In [ ]:
using Plots

In [ ]:
s = 10.;
b = 8/3.;
r = 28.;
mutable struct Lorenz
     x; y; z;
end
rLorenz()=Lorenz(50.0*rand()-25.,50.0*rand()-25.,50.0*rand())
function step!(a::Lorenz,x,dt)
    a.x+=dt*s*(a.y-x);
    a.y+=dt*(x*(r-a.z)-a.y);
    a.z+=dt*(x*a.y-b*a.z);
    end;
Base.copy(a::Lorenz)=Lorenz(a.x,a.y,a.z)

In [ ]:
#=
time step of array of attractors where x-coordinates 
are replaced in the derivatives by a weighted average of other x coordinates + noise
=#
function step!(attractors::Array{Lorenz,1},weights::Array{Float64,2},dt,rnd)
    #calculate new x-coordinates
    local xs=zeros(length(attractors));
    for i in 1:length(attractors)
        for j in 1:length(attractors)
            xs[i]+=weights[i,j]*attractors[j].x;
        end
        xs[i]/=sum(weights[i,:]);
        xs[i]+=2*rnd*(rand()-.5)
    end
    #update coordinates and perform time step
    for i in 1:length(attractors)
        step!(attractors[i],xs[i],dt);
    end
end

In [ ]:
# 3d plot of array of Lorenz attractors
function plotL(rec::Array{Array{Lorenz,1},1})
    x=[rec[i][1].x for i=1:length(rec)];
    y=[rec[i][1].y for i=1:length(rec)];
    z=[rec[i][1].z for i=1:length(rec)];
    for j=2:length(rec[1])
        x=hcat(x,[rec[i][j].x for i=1:length(rec)]);
        y=hcat(y,[rec[i][j].y for i=1:length(rec)]);
        z=hcat(z,[rec[i][j].z for i=1:length(rec)]);
    end
    plot3d(x,y,z, xlim=(-25,25), ylim=(-25,25), zlim=(0,50),leg=false)
end
# YZ plane plot of array of Lorenz attractors
function plotLyz(rec::Array{Array{Lorenz,1},1})
    y=[rec[i][1].y for i=1:length(rec)];
    z=[rec[i][1].z for i=1:length(rec)];
    for j=2:length(rec[1])
        y=hcat(y,[rec[i][j].y for i=1:length(rec)]);
        z=hcat(z,[rec[i][j].z for i=1:length(rec)]);
    end
    plot(y,z, xlim=(-25,25), ylim=(0,50),leg=false)
end
function distYZ(a::Lorenz,b::Lorenz)
    sqrt((a.y-b.y)^2+(a.z-b.z)^2)
end
function distX(a::Lorenz,b::Lorenz)
    abs(a.x-b.x)
end

In [ ]:
w1=[2. 1.;1. 2.];# synchro
w2=[1. 0.;0. 1.];# no synchro
dt=.01;
noise=.1^10;
as=Array{Lorenz,1}();
for i=1:2
    push!(as,rLorenz());
end
record=Array{Array{Lorenz,1},1}();
for j=1:3000
    step!(as,w1,dt,noise);
    push!(record,deepcopy(as));
end
for j=1:4000
    step!(as,w2,dt,noise);
    push!(record,deepcopy(as));
end
for j=1:3000
    step!(as,w1,dt,noise);
    push!(record,deepcopy(as));
end

In [ ]:
plot([record[i][1].x for i=1:length(record)],leg=false)

In [ ]:
plot([distYZ(record[i][1],record[i][2]) for i=1:length(record)],yaxis=:log,leg=false)

In [ ]:
plot([distX(record[i][1],record[i][2]) for i=1:length(record)],yaxis=:log,leg=false)